In [ ]:
import pandas as pd
from sqlalchemy import case
from sqlalchemy import literal
from sqlalchemy.orm import Query

import src
import src.db.models.bert_data as bm
import src.db.models.open_discourse as od
import src.pop_aggregate as pop_metrics

In [ ]:
pd.set_option("display.max_colwidth", 2048)
pd.set_option("display.max_rows", 256)

engine = src.db.connect.make_engine("DB")

# Load data


In [ ]:
thresh = {"elite": 0.415961, "centr": 0.295400, "left": 0.429109, "right": 0.302714}

In [ ]:
colormap = {
    "CDU/CSU": "#000000",
    "Grüne": "#1AA037",
    "DIE LINKE": "#8B008B",
    "FDP": "#FFEF00",
    "AfD": "#0489DB",
    "SPD": "#E3000F",
}

In [ ]:
query = (
    Query(bm.Sample)
    .join(bm.Prediction)
    .join(od.Speech)
    .join(od.Faction)
    .join(od.Politician)
    .filter(
        bm.Sample.sentence_length > 2,
        od.Faction.abbreviation != "Fraktionslos",
    )
    .with_entities(
        od.Speech.id.label("speech_id"),
        od.Speech.electoral_term,
        od.Speech.date,
        bm.Sample.sentence_no,
        od.Politician.id.label("pol_id"),
        od.Politician.first_name,
        od.Politician.last_name,
        case(
            (od.Faction.abbreviation == "DIE LINKE.", literal("DIE LINKE")),
            else_=od.Faction.abbreviation,
        ).label("abbreviation"),
        case(
            (bm.Prediction.elite >= thresh["elite"], literal(1)),
            (bm.Prediction.elite < thresh["elite"], literal(0)),
        ).label("antielite"),
        case(
            (bm.Prediction.pplcentr >= thresh["centr"], literal(1)),
            (bm.Prediction.pplcentr < thresh["centr"], literal(0)),
        ).label("pplcentr"),
        case(
            (bm.Prediction.left >= thresh["left"], literal(1)),
            (bm.Prediction.left < thresh["left"], literal(0)),
        ).label("left"),
        case(
            (bm.Prediction.right >= thresh["right"], literal(1)),
            (bm.Prediction.right < thresh["right"], literal(0)),
        ).label("right"),
    )
)

In [ ]:
with engine.connect() as conn:
    df_raw = pd.read_sql(query.statement, conn)
df_raw.date = pd.to_datetime(df_raw.date)

In [ ]:
# filter all speeches with less than 3 sentences
df_raw = df_raw.groupby("speech_id").filter(lambda x: x["sentence_no"].max() > 3)

In [ ]:
assert len(df_raw) == df_raw.groupby(["speech_id", "sentence_no"]).ngroups

# Basic sentence level stats


In [ ]:
len(df_raw)

1258876

In [ ]:
(df_raw.iloc[:, 8:].sum() / len(df_raw) * 100).round(1)

antielite    7.4
pplcentr     2.1
left         1.2
right        0.5
dtype: float64

In [ ]:
pop_total = df_raw["antielite"] * df_raw["pplcentr"]
pop_total = pop_total.apply(lambda x: 0 if x == 0 else 1)
print(pop_total.value_counts(normalize=False))
pop_total.value_counts(normalize=True).round(3)

0    1256473
1       2403
dtype: int64


0    0.998
1    0.002
dtype: float64

# Populism by Politician


## define funcs


In [ ]:
def show_output(df, var):
    most_populist_pols = []
    for _, group in df.groupby("electoral_term"):
        out = group.sort_values(var, ascending=False)
        out["rank"] = list(range(1, len(out) + 1))
        most_populist_pols.append(out.head(5))
    return pd.concat(most_populist_pols)

In [ ]:
def format_latex(tex):
    lines = tex.splitlines()
    new = []
    for i, line in enumerate(lines):
        # print(i, line)
        if i == 2:
            line = r"\headrow Term & Rank & First Name & Last Name & Party \\"
        if i == 3:
            continue
        if i == 5:
            line = line.replace(r"\multirow[c]{5}{*}{18}", r"\textbf{18}")
        if i == 10:
            line = line.replace(r"\multirow[c]{5}{*}{19}", r"\textbf{19}")
        if i == 10:
            new.append(r"& & & & \\\midrule")
        if 4 < i < 14:
            line = line + r"\midrule"
        new.append(line)

    tex = "\n".join(new)
    print(tex)

## Populism by electoral_term and politician


In [ ]:
df_pol = (
    df_raw.drop(["speech_id", "sentence_no", "date"], axis=1)
    .groupby(["pol_id", "first_name", "last_name", "abbreviation", "electoral_term"])
    .mean()
    .reset_index()
)

df_pol = df_pol.loc[df_pol.pol_id != -1, :]
df_pol["pop"] = df_pol.antielite * df_pol.pplcentr
df_pol["pop_left"] = df_pol["pop"] * df_pol.left
df_pol["pop_right"] = df_pol["pop"] * df_pol.right

pop_metrics.ELITE = "antielite"
pop_metrics.CENTR = "pplcentr"

df_pol["pop_bollen"] = df_pol.apply(lambda x: pop_metrics.bollen_index(x), axis=1)
df_pol["pop_goertz"] = df_pol.apply(lambda x: pop_metrics.goertz_index(x), axis=1)
thresh = {
    "antielite": df_pol.antielite.quantile(0.75),
    "pplcentr": df_pol.pplcentr.quantile(0.75),
}
df_pol["pop_sartori"] = df_pol.apply(
    lambda x: pop_metrics.sartori_index(x, threshold=thresh), axis=1
)

### general populism

-> use this for table in paper

In [ ]:
show_output(df_pol, "pop")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,pop_bollen,pop_goertz,pop_sartori,rank
646,11004183,Sahra,Wagenknecht,DIE LINKE,18,0.324531,0.039319,0.137911,0.013498,0.012760,0.001760,0.000172,0.181925,0.039319,1,1
261,11003625,Andreas,Scheuer,CDU/CSU,18,0.114094,0.067114,0.006711,0.013423,0.007657,0.000051,0.000103,0.090604,0.067114,1,2
340,11003786,Katja,Kipping,DIE LINKE,18,0.113697,0.054521,0.041888,0.003324,0.006199,0.000260,0.000021,0.084109,0.054521,1,3
410,11003869,Sabine,Zimmermann,DIE LINKE,18,0.120971,0.051068,0.047447,0.001811,0.006178,0.000293,0.000011,0.086020,0.051068,1,4
115,11003132,Katrin Dagmar,Göring-Eckardt,Grüne,18,0.134140,0.043369,0.026475,0.001765,0.005817,0.000154,0.000010,0.088754,0.043369,1,5
1278,11004892,Martin,Sichert,AfD,19,0.308852,0.113443,0.051803,0.080656,0.035037,0.001815,0.002826,0.211148,0.113443,1,1
1309,11004930,Alice,Weidel,AfD,19,0.375191,0.061256,0.054364,0.061256,0.022983,0.001249,0.001408,0.218224,0.061256,1,2
1332,11004962,Janosch,Dahmen,Grüne,19,0.321429,0.071429,0.000000,0.000000,0.022959,0.000000,0.000000,0.196429,0.071429,1,3
1250,11004859,Martin,Reichardt,AfD,19,0.259629,0.078459,0.019971,0.074180,0.020370,0.000407,0.001511,0.169044,0.078459,1,4
1153,11004749,Martin,Hess,AfD,19,0.318713,0.059211,0.003655,0.097953,0.018871,0.000069,0.001848,0.188962,0.059211,1,5


In [ ]:
df = show_output(df_pol, "pop")

df = (
    df.rename({"abbreviation": "party"}, axis=1)
    .loc[:, ["electoral_term", "rank", "first_name", "last_name", "party"]]
    .set_index(["electoral_term", "rank"])
)

tex = df.style.to_latex(hrules=True)

format_latex(tex)

\begin{tabular}{lllll}
\toprule
\headrow Term & Rank & First Name & Last Name & Party \\
\midrule
\textbf{18} & 1 & Sahra & Wagenknecht & DIE LINKE \\\midrule
 & 2 & Andreas & Scheuer & CDU/CSU \\\midrule
 & 3 & Katja & Kipping & DIE LINKE \\\midrule
 & 4 & Sabine & Zimmermann & DIE LINKE \\\midrule
 & 5 & Katrin Dagmar & Göring-Eckardt & Grüne \\\midrule
& & & & \\\midrule
\textbf{19} & 1 & Martin & Sichert & AfD \\\midrule
 & 2 & Alice & Weidel & AfD \\\midrule
 & 3 & Janosch & Dahmen & Grüne \\\midrule
 & 4 & Martin & Reichardt & AfD \\\midrule
 & 5 & Martin & Hess & AfD \\
\bottomrule
\end{tabular}


### Goertz Index


In [ ]:
show_output(df_pol, "pop_goertz")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,pop_bollen,pop_goertz,pop_sartori,rank
261,11003625,Andreas,Scheuer,CDU/CSU,18,0.114094,0.067114,0.006711,0.013423,0.007657,0.000051,0.000103,0.090604,0.067114,1,1
340,11003786,Katja,Kipping,DIE LINKE,18,0.113697,0.054521,0.041888,0.003324,0.006199,0.000260,0.000021,0.084109,0.054521,1,2
410,11003869,Sabine,Zimmermann,DIE LINKE,18,0.120971,0.051068,0.047447,0.001811,0.006178,0.000293,0.000011,0.086020,0.051068,1,3
921,11004390,Corinna,Rüffer,Grüne,18,0.062581,0.050847,0.020860,0.000000,0.003182,0.000066,0.000000,0.056714,0.050847,0,4
699,11004256,Karl Heinz,Brunner,SPD,18,0.052221,0.048324,0.003118,0.003118,0.002524,0.000008,0.000008,0.050273,0.048324,0,5
1278,11004892,Martin,Sichert,AfD,19,0.308852,0.113443,0.051803,0.080656,0.035037,0.001815,0.002826,0.211148,0.113443,1,1
1029,11004454,Pia,Zimmermann,DIE LINKE,19,0.125506,0.097166,0.066802,0.000000,0.012195,0.000815,0.000000,0.111336,0.097166,1,2
341,11003786,Katja,Kipping,DIE LINKE,19,0.130435,0.092995,0.072464,0.002415,0.012130,0.000879,0.000029,0.111715,0.092995,1,3
295,11003715,Lars,Klingbeil,SPD,19,0.111650,0.087379,0.014563,0.000000,0.009756,0.000142,0.000000,0.099515,0.087379,1,4
1250,11004859,Martin,Reichardt,AfD,19,0.259629,0.078459,0.019971,0.074180,0.020370,0.000407,0.001511,0.169044,0.078459,1,5


In [ ]:
df = show_output(df_pol, "pop_goertz")

df = (
    df.rename({"abbreviation": "party"}, axis=1)
    .loc[:, ["electoral_term", "rank", "first_name", "last_name", "party"]]
    .set_index(["electoral_term", "rank"])
)

tex = df.style.to_latex(hrules=True)

format_latex(tex)

\begin{tabular}{lllll}
\toprule
\headrow Term & Rank & First Name & Last Name & Party \\
\midrule
\textbf{18} & 1 & Andreas & Scheuer & CDU/CSU \\\midrule
 & 2 & Katja & Kipping & DIE LINKE \\\midrule
 & 3 & Sabine & Zimmermann & DIE LINKE \\\midrule
 & 4 & Corinna & Rüffer & Grüne \\\midrule
 & 5 & Karl Heinz & Brunner & SPD \\\midrule
& & & & \\\midrule
\textbf{19} & 1 & Martin & Sichert & AfD \\\midrule
 & 2 & Pia & Zimmermann & DIE LINKE \\\midrule
 & 3 & Katja & Kipping & DIE LINKE \\\midrule
 & 4 & Lars & Klingbeil & SPD \\\midrule
 & 5 & Martin & Reichardt & AfD \\
\bottomrule
\end{tabular}


### Bollen Index


In [ ]:
show_output(df_pol, "pop_bollen")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,pop_bollen,pop_goertz,pop_sartori,rank
5,11000198,Peter,Bleser,CDU/CSU,18,0.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0,1
646,11004183,Sahra,Wagenknecht,DIE LINKE,18,0.324531,0.039319,0.137911,0.013498,0.012760,0.001760,0.000172,0.181925,0.039319,1,2
10,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE,18,0.273535,0.019538,0.115453,0.010657,0.005344,0.000617,0.000057,0.146536,0.019538,0,3
309,11003746,Sevim,Dağdelen,DIE LINKE,18,0.254300,0.019991,0.015342,0.018131,0.005084,0.000078,0.000092,0.137146,0.019991,0,4
400,11003858,Alexander,Ulrich,DIE LINKE,18,0.210031,0.015674,0.047022,0.001567,0.003292,0.000155,0.000005,0.112853,0.015674,0,5
1309,11004930,Alice,Weidel,AfD,19,0.375191,0.061256,0.054364,0.061256,0.022983,0.001249,0.001408,0.218224,0.061256,1,1
1278,11004892,Martin,Sichert,AfD,19,0.308852,0.113443,0.051803,0.080656,0.035037,0.001815,0.002826,0.211148,0.113443,1,2
1251,11004862,Martin Erwin,Renner,AfD,19,0.387342,0.010127,0.012658,0.136709,0.003922,0.000050,0.000536,0.198734,0.010127,0,3
1332,11004962,Janosch,Dahmen,Grüne,19,0.321429,0.071429,0.000000,0.000000,0.022959,0.000000,0.000000,0.196429,0.071429,1,4
1153,11004749,Martin,Hess,AfD,19,0.318713,0.059211,0.003655,0.097953,0.018871,0.000069,0.001848,0.188962,0.059211,1,5


In [ ]:
df = show_output(df_pol, "pop_bollen")

df = (
    df.rename({"abbreviation": "party"}, axis=1)
    .loc[:, ["electoral_term", "rank", "first_name", "last_name", "party"]]
    .set_index(["electoral_term", "rank"])
)

tex = df.style.to_latex(hrules=True)

format_latex(tex)

\begin{tabular}{lllll}
\toprule
\headrow Term & Rank & First Name & Last Name & Party \\
\midrule
\textbf{18} & 1 & Peter & Bleser & CDU/CSU \\\midrule
 & 2 & Sahra & Wagenknecht & DIE LINKE \\\midrule
 & 3 & Jörg Diether & Dehm-Desoi & DIE LINKE \\\midrule
 & 4 & Sevim & Dağdelen & DIE LINKE \\\midrule
 & 5 & Alexander & Ulrich & DIE LINKE \\\midrule
& & & & \\\midrule
\textbf{19} & 1 & Alice & Weidel & AfD \\\midrule
 & 2 & Martin & Sichert & AfD \\\midrule
 & 3 & Martin Erwin & Renner & AfD \\\midrule
 & 4 & Janosch & Dahmen & Grüne \\\midrule
 & 5 & Martin & Hess & AfD \\
\bottomrule
\end{tabular}


### Sartori Index


In [ ]:
show_output(df_pol, "pop_sartori")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,pop_bollen,pop_goertz,pop_sartori,rank
676,11004243,Luise,Amtsberg,Grüne,18,0.095204,0.037938,0.010021,0.002147,0.003612,0.000036,0.000008,0.066571,0.037938,1,1
115,11003132,Katrin Dagmar,Göring-Eckardt,Grüne,18,0.134140,0.043369,0.026475,0.001765,0.005817,0.000154,0.000010,0.088754,0.043369,1,2
453,11004030,Katja,Dörner,Grüne,18,0.095874,0.040049,0.018204,0.000607,0.003840,0.000070,0.000002,0.067961,0.040049,1,3
1002,11004439,Albert,Weiler,CDU/CSU,18,0.111772,0.032105,0.013080,0.010702,0.003588,0.000047,0.000038,0.071938,0.032105,1,4
1010,11004443,Kai,Whittaker,CDU/CSU,18,0.121103,0.041966,0.016787,0.010791,0.005082,0.000085,0.000055,0.081535,0.041966,1,5
1332,11004962,Janosch,Dahmen,Grüne,19,0.321429,0.071429,0.000000,0.000000,0.022959,0.000000,0.000000,0.196429,0.071429,1,1
1101,11004695,Tino,Chrupalla,AfD,19,0.174944,0.037064,0.016308,0.018532,0.006484,0.000106,0.000120,0.106004,0.037064,1,2
1249,11004858,Stephan,Protschka,AfD,19,0.209195,0.040230,0.014943,0.028736,0.008416,0.000126,0.000242,0.124713,0.040230,1,3
1250,11004859,Martin,Reichardt,AfD,19,0.259629,0.078459,0.019971,0.074180,0.020370,0.000407,0.001511,0.169044,0.078459,1,4
1254,11004865,Bernd,Riexinger,DIE LINKE,19,0.173203,0.078431,0.111111,0.003268,0.013585,0.001509,0.000044,0.125817,0.078431,1,5


In [ ]:
df_pol[["pop", "pop_bollen", "pop_goertz"]].corr(method="spearman")

,pop,pop_bollen,pop_goertz
pop,1.000000,0.866854,0.843890
pop_bollen,0.866854,1.000000,0.552324
pop_goertz,0.843890,0.552324,1.000000


### left-wing populism


In [ ]:
show_output(df_pol, "pop_left")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,rank
646,11004183,Sahra,Wagenknecht,DIE LINKE.,18,0.309859,0.048709,0.119718,0.023474,0.015093,0.001807,0.000354,1
340,11003786,Katja,Kipping,DIE LINKE.,18,0.118351,0.085106,0.064495,0.007314,0.010072,0.000650,0.000074,2
10,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE.,18,0.232682,0.026643,0.095915,0.021314,0.006199,0.000595,0.000132,3
410,11003869,Sabine,Zimmermann,DIE LINKE.,18,0.121333,0.073886,0.055053,0.002898,0.008965,0.000494,0.000026,4
588,11004129,Richard,Pitterle,DIE LINKE.,18,0.156664,0.025721,0.080281,0.000000,0.004030,0.000323,0.000000,5
1254,11004865,Bernd,Riexinger,DIE LINKE.,19,0.189542,0.114379,0.124183,0.003268,0.021680,0.002692,0.000071,1
1278,11004892,Martin,Sichert,AfD,19,0.315410,0.130492,0.050492,0.112131,0.041158,0.002078,0.004615,2
647,11004183,Sahra,Wagenknecht,DIE LINKE.,19,0.303333,0.056667,0.103333,0.008333,0.017189,0.001776,0.000143,3
1231,11004837,Zaklin,Nastic,DIE LINKE.,19,0.257340,0.063903,0.107081,0.024180,0.016445,0.001761,0.000398,4
341,11003786,Katja,Kipping,DIE LINKE.,19,0.132850,0.130435,0.088164,0.003623,0.017328,0.001528,0.000063,5


### right-wing populism


In [ ]:
show_output(df_pol, "pop_right")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,rank
646,11004183,Sahra,Wagenknecht,DIE LINKE.,18,0.309859,0.048709,0.119718,0.023474,0.015093,0.001807,0.000354,1
261,11003625,Andreas,Scheuer,CDU/CSU,18,0.114094,0.073826,0.000000,0.033557,0.008423,0.000000,0.000283,2
309,11003746,Sevim,Dağdelen,DIE LINKE.,18,0.243608,0.028359,0.022780,0.034868,0.006908,0.000157,0.000241,3
10,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE.,18,0.232682,0.026643,0.095915,0.021314,0.006199,0.000595,0.000132,4
340,11003786,Katja,Kipping,DIE LINKE.,18,0.118351,0.085106,0.064495,0.007314,0.010072,0.000650,0.000074,5
1278,11004892,Martin,Sichert,AfD,19,0.315410,0.130492,0.050492,0.112131,0.041158,0.002078,0.004615,1
1104,11004698,Gottfried,Curio,AfD,19,0.283436,0.075617,0.006173,0.155350,0.021433,0.000132,0.003330,2
1309,11004930,Alice,Weidel,AfD,19,0.389740,0.073507,0.035988,0.079632,0.028649,0.001031,0.002281,3
1250,11004859,Martin,Reichardt,AfD,19,0.267475,0.093438,0.027104,0.086305,0.024992,0.000677,0.002157,4
1153,11004749,Martin,Hess,AfD,19,0.293129,0.061404,0.005117,0.112573,0.017999,0.000092,0.002026,5


## Populism by Policitian + speech + electoral_term


In [ ]:
df_speeches = (
    df_raw.drop(["sentence_no", "date"], axis=1)
    .groupby(
        [
            "pol_id",
            "first_name",
            "last_name",
            "abbreviation",
            "electoral_term",
            "speech_id",
        ]
    )
    .mean()
    .reset_index()
)
df_speeches = df_speeches.loc[df_speeches.pol_id != -1, :]
df_speeches["pop"] = df_speeches.antielite * df_speeches.pplcentr
df_speeches["pop_left"] = df_speeches["pop"] * df_speeches.left
df_speeches["pop_right"] = df_speeches["pop"] * df_speeches.right

df_pol = (
    df_speeches.drop("speech_id", axis=1)
    .groupby(["pol_id", "first_name", "last_name", "abbreviation", "electoral_term"])
    .mean()
    .reset_index()
)

### general populism


In [ ]:
show_output(df_pol, "pop")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,rank
641,11004183,Sahra,Wagenknecht,DIE LINKE.,18,0.274289,0.039201,0.103937,0.025621,0.012490,0.001786,0.000593,1
256,11003625,Andreas,Scheuer,CDU/CSU,18,0.112495,0.076862,0.000000,0.034879,0.008239,0.000000,0.000436,2
405,11003869,Sabine,Zimmermann,DIE LINKE.,18,0.104987,0.073413,0.054851,0.001975,0.007884,0.000604,0.000013,3
16,11001023,Ulla,Jelpke,DIE LINKE.,18,0.143658,0.042774,0.038165,0.013965,0.007573,0.000738,0.000087,4
5,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE.,18,0.203880,0.020538,0.074360,0.021962,0.007298,0.001571,0.000161,5
1273,11004892,Martin,Sichert,AfD,19,0.270665,0.114663,0.044342,0.103656,0.034284,0.002769,0.005539,1
1304,11004930,Alice,Weidel,AfD,19,0.379145,0.072685,0.032144,0.073528,0.030785,0.001175,0.002545,2
1135,11004736,Mariana Iris,Harder-Kühnel,AfD,19,0.198616,0.102246,0.029713,0.100301,0.026049,0.003297,0.004667,3
1245,11004859,Martin,Reichardt,AfD,19,0.259349,0.086581,0.023207,0.081402,0.023748,0.001040,0.003140,4
1249,11004865,Bernd,Riexinger,DIE LINKE.,19,0.176643,0.109654,0.115164,0.003367,0.023342,0.004271,0.000291,5


### left-wing populism


In [ ]:
show_output(df_pol, "pop_left")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,rank
323,11003773,Inge,Höger-Neuling,DIE LINKE.,18,0.091668,0.013452,0.025414,0.003997,0.005170,0.002857,0.000029,1
641,11004183,Sahra,Wagenknecht,DIE LINKE.,18,0.274289,0.039201,0.103937,0.025621,0.012490,0.001786,0.000593,2
5,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE.,18,0.203880,0.020538,0.074360,0.021962,0.007298,0.001571,0.000161,3
395,11003858,Alexander,Ulrich,DIE LINKE.,18,0.212245,0.036868,0.044595,0.014548,0.007054,0.001110,0.000144,4
207,11003556,Gabriele,Hiller-Ohm,SPD,18,0.038301,0.041432,0.026086,0.000000,0.004176,0.000833,0.000000,5
380,11003837,Gerhard,Schick,Grüne,19,0.163133,0.042694,0.106275,0.000000,0.013360,0.004554,0.000000,1
336,11003786,Katja,Kipping,DIE LINKE.,19,0.114209,0.122394,0.079609,0.002787,0.020963,0.004305,0.000177,2
1249,11004865,Bernd,Riexinger,DIE LINKE.,19,0.176643,0.109654,0.115164,0.003367,0.023342,0.004271,0.000291,3
1135,11004736,Mariana Iris,Harder-Kühnel,AfD,19,0.198616,0.102246,0.029713,0.100301,0.026049,0.003297,0.004667,4
6,11000365,Jörg Diether,Dehm-Desoi,DIE LINKE.,19,0.169962,0.042400,0.048584,0.014086,0.012899,0.003209,0.000285,5


### right-wing populism


In [ ]:
show_output(df_pol, "pop_right")

,pol_id,first_name,last_name,abbreviation,electoral_term,antielite,pplcentr,left,right,pop,pop_left,pop_right,rank
213,11003572,Michael,Kretschmer,CDU/CSU,18,0.043029,0.044400,0.004167,0.014113,0.007278,0.000058,0.001563,1
641,11004183,Sahra,Wagenknecht,DIE LINKE.,18,0.274289,0.039201,0.103937,0.025621,0.012490,0.001786,0.000593,2
838,11004346,Yvonne,Magwas,CDU/CSU,18,0.059188,0.032314,0.003559,0.021350,0.004102,0.000001,0.000453,3
256,11003625,Andreas,Scheuer,CDU/CSU,18,0.112495,0.076862,0.000000,0.034879,0.008239,0.000000,0.000436,4
304,11003746,Sevim,Dağdelen,DIE LINKE.,18,0.162156,0.022724,0.016529,0.024319,0.004294,0.000433,0.000223,5
1107,11004707,Thomas,Ehrhorn,AfD,19,0.246710,0.050332,0.012318,0.130780,0.020296,0.000206,0.009280,1
1099,11004698,Gottfried,Curio,AfD,19,0.269928,0.061637,0.003636,0.151902,0.022402,0.000102,0.006492,2
1273,11004892,Martin,Sichert,AfD,19,0.270665,0.114663,0.044342,0.103656,0.034284,0.002769,0.005539,3
1135,11004736,Mariana Iris,Harder-Kühnel,AfD,19,0.198616,0.102246,0.029713,0.100301,0.026049,0.003297,0.004667,4
1245,11004859,Martin,Reichardt,AfD,19,0.259349,0.086581,0.023207,0.081402,0.023748,0.001040,0.003140,5
